## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data

X= ['Age', 'PercentSalaryHike', 'DistanceFromHome', 'Education', 'YearsAtCompany', 'YearsInCurrentRole', 'HourlyRate', 'EducationField', 'JobSatisfaction', 'NumCompaniesWorked']

# Create X_df using your selected columns
X_df = attrition_df[X]
X_df.head()

# Show the data types for X_df
print(X_df.dtypes)


Age                    int64
PercentSalaryHike      int64
DistanceFromHome       int64
Education              int64
YearsAtCompany         int64
YearsInCurrentRole     int64
HourlyRate             int64
EducationField        object
JobSatisfaction        int64
NumCompaniesWorked     int64
dtype: object


In [5]:
from re import X
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=78)


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
from sklearn.preprocessing import OneHotEncoder

education_encoder = OneHotEncoder(sparse_output=False)

education_encoded_train = education_encoder.fit_transform(X_train[['EducationField']])
education_encoded_test = education_encoder.transform(X_test[['EducationField']])
#get feature names out

education_columns = education_encoder.get_feature_names_out(['EducationField'])

#create new dataframe
education_encoded_train_df = pd.DataFrame(education_encoded_train, columns=education_columns)
education_encoded_test_df = pd.DataFrame(education_encoded_test, columns=education_columns)
X_train_encoded = pd.concat([X_train.drop('EducationField', axis =1), education_encoded_train_df], axis=1)
X_test_encoded = pd.concat([X_test.drop('EducationField', axis =1), education_encoded_test_df], axis=1)

education_encoded = np.concatenate((education_encoded_train, education_encoded_test), axis=0)


education_encoded_df = pd.DataFrame(education_encoded, columns=education_columns)

education_encoded_df.head()



,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree
0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
# Create a StandardScaler
scaler = StandardScaler()




# Fit the StandardScaler to the training data
X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train_encoded.select_dtypes(include='number')),
                columns=X_train_encoded.select_dtypes(include='number').columns,
                index=X_train_encoded.index)
X_test_scaled = pd.DataFrame(
                scaler.fit_transform(X_test_encoded.select_dtypes(include='number')),
                columns=X_test_encoded.select_dtypes(include='number').columns,
                index=X_test_encoded.index)

# Scale the training and testing data
X_train_encoded = pd.concat([X_train_scaled, X_train_encoded.select_dtypes(exclude='number')], axis=1)
X_test_encoded = pd.concat([X_test_scaled, X_test_encoded.select_dtypes(exclude='number')], axis=1)

print(X_train_encoded.head())
print(X_test_encoded.head())


           Age  PercentSalaryHike  DistanceFromHome  Education  \
1304  0.357157          -0.863925          0.377850   0.098427   
1125 -0.864144           1.057672         -0.879869  -1.863440   
139  -0.753117           1.057672          0.000535   0.098427   
499  -0.420035          -0.863925         -0.125237   1.079361   
832   0.024075          -0.589411          2.012885  -0.882507   

      YearsAtCompany  YearsInCurrentRole  HourlyRate  JobSatisfaction  \
1304        0.018755           -0.048758   -0.957281        -1.555928   
1125       -0.147079           -0.048758    1.016270         1.152169   
139         0.682093            1.323006   -0.907943         1.152169   
499        -0.312914           -1.146170   -1.351992         0.249470   
832        -0.147079           -0.597464   -0.710588         1.152169   

      NumCompaniesWorked  EducationField_Human Resources  \
1304           -0.275838                             NaN   
1125           -0.676808                    

In [9]:
# Create a OneHotEncoder for the Department column

department_encoder = OneHotEncoder(sparse_output=False)
# Fit the encoder to the training data
department_encoded_train = department_encoder.fit_transform(y_train[['Department']])
department_encoded_test = department_encoder.transform(y_test[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
encoded_columns = department_encoder.get_feature_names_out(['Department'])
department_encoded_train_df = pd.DataFrame(department_encoded_train, columns=encoded_columns, index=y_train.index)
department_encoded_test_df = pd.DataFrame(department_encoded_test, columns=encoded_columns, index=y_test.index)

X_train_encoded = pd.concat([X_train_encoded, department_encoded_train_df], axis=1)
X_test_encoded = pd.concat([X_test_encoded, department_encoded_test_df], axis=1)

print(X_train_encoded.head())
print(X_test_encoded.head())


           Age  PercentSalaryHike  DistanceFromHome  Education  \
1304  0.357157          -0.863925          0.377850   0.098427   
1125 -0.864144           1.057672         -0.879869  -1.863440   
139  -0.753117           1.057672          0.000535   0.098427   
499  -0.420035          -0.863925         -0.125237   1.079361   
832   0.024075          -0.589411          2.012885  -0.882507   

      YearsAtCompany  YearsInCurrentRole  HourlyRate  JobSatisfaction  \
1304        0.018755           -0.048758   -0.957281        -1.555928   
1125       -0.147079           -0.048758    1.016270         1.152169   
139         0.682093            1.323006   -0.907943         1.152169   
499        -0.312914           -1.146170   -1.351992         0.249470   
832        -0.147079           -0.597464   -0.710588         1.152169   

      NumCompaniesWorked  EducationField_Human Resources  \
1304           -0.275838                             NaN   
1125           -0.676808                    

In [10]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data

attrition_encoded_train = attrition_encoder.fit_transform(y_train[['Attrition']])
attrition_encoded_test = attrition_encoder.transform(y_test[['Attrition']])
# Create two new variables by applying the encoder
# to the training and testing data

encoded_columns = attrition_encoder.get_feature_names_out(['Attrition'])
attrition_encoded_train_df = pd.DataFrame(attrition_encoded_train, columns=encoded_columns, index=y_train.index)
attrition_encoded_test_df = pd.DataFrame(attrition_encoded_test, columns=encoded_columns, index=y_test.index)

X_train_encoded = pd.concat([X_train_encoded, attrition_encoded_train_df], axis=1)
X_test_encoded = pd.concat([X_test_encoded, attrition_encoded_test_df], axis=1)

print(X_train_encoded.head())
print(X_test_encoded.head())

           Age  PercentSalaryHike  DistanceFromHome  Education  \
1304  0.357157          -0.863925          0.377850   0.098427   
1125 -0.864144           1.057672         -0.879869  -1.863440   
139  -0.753117           1.057672          0.000535   0.098427   
499  -0.420035          -0.863925         -0.125237   1.079361   
832   0.024075          -0.589411          2.012885  -0.882507   

      YearsAtCompany  YearsInCurrentRole  HourlyRate  JobSatisfaction  \
1304        0.018755           -0.048758   -0.957281        -1.555928   
1125       -0.147079           -0.048758    1.016270         1.152169   
139         0.682093            1.323006   -0.907943         1.152169   
499        -0.312914           -1.146170   -1.351992         0.249470   
832        -0.147079           -0.597464   -0.710588         1.152169   

      NumCompaniesWorked  EducationField_Human Resources  \
1304           -0.275838                             NaN   
1125           -0.676808                    

## Create, Compile, and Train the Model

In [17]:
# Find the number of columns in the X training data

input_dim = X_train_encoded.shape[1]
# Create the input layer
input_layer = layers.Input(shape=(X_train_encoded.shape[1],), name='input_features')


# Create at least two shared layers
shared_layer_1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer_2 = layers.Dense(32, activation='relu')(shared_layer_1)

In [19]:
# Create a branch for Department
# with a hidden layer and an output layer
department_hidden = layers.Dense(64, activation='relu')(shared_layer_2)
department_output = layers.Dense(department_encoded_train.shape[1], activation='softmax', name='department_output')(department_hidden)
# Create the hidden layer

# Create the output layer



In [29]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_output = layers.Dense(2, activation='softmax', name='attrition_output')(shared_layer_2)
# Create the hidden layer

attrition_hidden_layer = layers.Dense(64, activation='relu')(shared_layer_2)
# Create the output layer



In [33]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
              'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_features            │ (None, 20)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 64)             │          1,344 │ input_features[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 32)             │          2,080 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 64)             │          2,112 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │            195 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ dense_6[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,797 (22.64 KB)

 Trainable params: 5,797 (22.64 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# Train the model
history = model.fit(X_train_encoded[:40],
                    {'department_output': department_encoded_train,
                    'attrition_output': attrition_encoded_train},
                    epochs=50,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step - attrition_output_accuracy: 0.9375 - department_output_accuracy: 0.0312 - loss: nan - val_attrition_output_accuracy: 1.0000 - val_department_output_accuracy: 0.0000e+00 - val_loss: nan
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - attrition_output_accuracy: 0.9375 - department_output_accuracy: 0.0312 - loss: nan - val_attrition_output_accuracy: 1.0000 - val_department_output_accuracy: 0.0000e+00 - val_loss: nan
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - attrition_output_accuracy: 0.9375 - department_output_accuracy: 0.0312 - loss: nan - val_attrition_output_accuracy: 1.0000 - val_department_output_accuracy: 0.0000e+00 - val_loss: nan
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - attrition_output_accuracy: 0.9375 - department_output_accuracy: 0.0312 - loss: nan - val_attrition_output_accuracy: 1.0000 - val_department_output_accuracy: 0.0000e+00 - val_loss: nan
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - attrit

In [38]:
# Evaluate the model with the testing data
test_loss, department_accuracy, attrition_accuracy, = model.evaluate(X_test_encoded[:294],
                                                            {'department_output': department_encoded_test,
                                                            'attrition_output': attrition_encoded_test})

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8636 - department_output_accuracy: 0.0648 - loss: nan 


In [40]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {department_accuracy:.4f}")
print(f"Attrition Accuracy: {attrition_accuracy:.4f}")

Department Accuracy: 0.8571
Attrition Accuracy: 0.0544


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?
Accuracy may not be the optimal metric for this dataset. While it provides a quick overview of model performance, it can be misleading, especially if the classes are imbalanced. For instance, in employee attrition prediction, there's often a disproportionate number of non-attrition cases. In such scenarios, a model could achieve high accuracy by simply predicting the majority class, without truly understanding the underlying patterns.
2. What activation functions did you choose for your output layers, and why?

The code uses softmax activation for both department and attrition output layers. This choice is appropriate for multi-class classification problems. Softmax ensures the output probabilities sum to 1 across all classes, making it suitable for department prediction where multiple categories likely exist. For attrition, which is typically binary, softmax reduces to logistic sigmoid, still providing a valid probability distribution.


3. Can you name a few ways that this model might be improved?

Data preprocessing: The code slices the input data (X_train_encoded[:40] and X_test_encoded[:294]), which may limit the model's learning. Using the full dataset could potentially improve performance.

Hyperparameter tuning: Experimenting with different network architectures, learning rates, and batch sizes could lead to better results.

Feature engineering: Creating new features or transforming existing ones might help capture more complex relationships in the data.



YOUR ANSWERS HERE

1.
2.
3.